Set the environment variable for the Groq API key to authenticate requests.

In [1]:
from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

Install the necessary Python packages: `langchain-groq`, `langchain-core`, and `requests`.

In [2]:
!pip install -q langchain-groq==0.3.8 langchain-core==0.3.* requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.6 MB/s eta 0:00:00


In [3]:
!pip show langchain-groq langchain-core requests

Name: langchain-groq
Version: 0.3.8
Summary: An integration package connecting Groq and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: groq, langchain-core
Required-by: 
---
Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-groq, langchain-text-splitters
---
Name: requests
Version: 2.32.4
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: bigframes, blosc2, CacheControl, community, datasets, diffusers, earthengine-api, fastai, folium, 

Import the required modules from LangChain and the `requests` library.

In [4]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

Create a simple tool using the `@tool` decorator that multiplies two numbers.

In [5]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

Invoke the `multiply` tool with sample arguments to test its functionality.

In [6]:
print(multiply.invoke({'a':3, 'b':4}))

12


Display the name of the `multiply` tool.

In [7]:
multiply.name

'multiply'

Display the description of the `multiply` tool.

In [8]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

Display the argument schema for the `multiply` tool.

In [9]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

Create a ChatGroq language model instance with a specified model and temperature.

In [11]:
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0, api_key=GROQ_API_KEY)

Invoke the LLM with a simple greeting to test its response.

In [12]:
llm.invoke('hi')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hi". We need to respond. The instruction: "You are ChatGPT, a large language model trained by OpenAI." There\'s no special instruction. Just respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 72, 'total_tokens': 129, 'completion_time': 0.059084996, 'prompt_time': 0.004621918, 'queue_time': 0.003028036, 'total_time': 0.063706914}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_0e024b9980', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c90f42b6-91d4-4a99-8a0a-2109ec4ab41b-0', usage_metadata={'input_tokens': 72, 'output_tokens': 57, 'total_tokens': 129})

Bind the `multiply` tool to the LLM so it can use the tool during conversations.

In [13]:
llm_with_tools = llm.bind_tools([multiply])

Invoke the LLM with tools enabled using a simple greeting.

In [14]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="Hello! I'm just a bunch of code, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'reasoning_content': 'User says "Hi how are you". We respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 133, 'total_tokens': 186, 'completion_time': 0.052226061, 'prompt_time': 0.007458755, 'queue_time': 0.003085873, 'total_time': 0.059684816}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_d8063b0a5c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--209ea539-9f49-4c54-8f24-bcdb0d9e8a7a-0', usage_metadata={'input_tokens': 133, 'output_tokens': 53, 'total_tokens': 186})

Create a `HumanMessage` asking the LLM to multiply two numbers.

In [15]:
query = HumanMessage('can you multiply 3 with 1000')

Initialize a list of messages to be used in the conversation.

In [16]:
messages = [query]

Display the current list of messages.

In [17]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

Invoke the LLM with the prepared messages and tools.

In [18]:
result = llm_with_tools.invoke(messages)

Append the AI's response to the messages list.

In [19]:
messages.append(result)

Display the updated list of messages after appending the AI response.

In [20]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function.', 'tool_calls': [{'id': 'fc_aecc1fc3-9fd5-46fc-8d9d-6b60dd32a7ea', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 138, 'total_tokens': 174, 'completion_time': 0.03559666, 'prompt_time': 0.007738057, 'queue_time': 0.002861841, 'total_time': 0.043334717}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_d6de37e6be', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b7280f3e-09ff-4cd5-8f00-a4897547f70f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_aecc1fc3-9fd5-46fc-8d9d-6b60dd32a7ea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 138, 'output_tokens': 36, 'total_tokens': 174})]

Invoke the `multiply` tool using the arguments extracted from the AI's tool call.

In [21]:
tool_result = multiply.invoke(result.tool_calls[0])

Display the result returned by the `multiply` tool.

In [22]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='fc_aecc1fc3-9fd5-46fc-8d9d-6b60dd32a7ea')

Append the tool result to the messages list.

In [23]:
messages.append(tool_result)

Display the final list of messages after all steps.

In [24]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function.', 'tool_calls': [{'id': 'fc_aecc1fc3-9fd5-46fc-8d9d-6b60dd32a7ea', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 138, 'total_tokens': 174, 'completion_time': 0.03559666, 'prompt_time': 0.007738057, 'queue_time': 0.002861841, 'total_time': 0.043334717}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_d6de37e6be', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b7280f3e-09ff-4cd5-8f00-a4897547f70f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_aecc1fc3-9fd5-46fc-8d9d-6b60dd32a7ea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 138, 'output_tokens': 36, 'total_tokens': 174}),
 ToolM

Invoke the LLM with the full conversation and display its final response.

In [25]:
llm_with_tools.invoke(messages).content

'Sure! 3 × 1000 = **3000**.'

Define two tools: one to fetch the currency conversion factor using an API, and another to convert a value using the conversion rate.

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/[EXCHANGERATE_API_KEY]/pair/{base_currency}/{target_currency}' # Exchange Rate Api Key

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


Display the argument schema for the `convert` tool.

In [27]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

Invoke the `get_conversion_factor` tool with sample currencies to test its output.

In [25]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1761091202,
 'time_last_update_utc': 'Wed, 22 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1761177602,
 'time_next_update_utc': 'Thu, 23 Oct 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.0293}

Invoke the `convert` tool with sample values to test its output.

In [7]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

Bind the currency conversion tools to a new LLM instance.

In [7]:
# tool binding
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0, api_key=GROQ_API_KEY)

Bind the `get_conversion_factor` and `convert` tools to the LLM.

In [8]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

Create a `HumanMessage` asking for the conversion factor and to convert a value between INR and USD.

In [9]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

Display the current list of messages for the currency conversion task.

In [10]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

Invoke the LLM with the currency conversion query and tools.

In [11]:
ai_message = llm_with_tools.invoke(messages)

Append the AI's response to the messages list.

In [12]:
messages.append(ai_message)

Display the tool calls suggested by the AI in its response.

In [13]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'fc_e40fefed-7d20-4748-b8c7-eb115bbe4a9c',
  'type': 'tool_call'}]

Iterate through the tool calls, execute them, and update the messages list with the results.

In [14]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



Display the updated list of messages after executing the tool calls.

In [15]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the function get_conversion_factor to get factor between INR and USD. Then use convert to convert 10 INR to USD. The convert function expects base_currency_value: number. But we need to specify base_currency? The function signature: convert(_:{base_currency_value: number}) => any. It doesn\'t specify target currency. Maybe the context is that the conversion factor is known from previous function. But we need to call get_conversion_factor first. Then call convert with base_currency_value: 10. But we need to pass the conversion factor? The convert function likely uses the conversion factor from the previous call. But we don\'t know how to pass it. The function signature is incomplete. But we can call get_conversion_factor with base_currency: "I

Invoke the LLM with the full conversation and display its final response for the currency conversion task.

In [16]:
llm_with_tools.invoke(messages).content

'**Conversion factor (INR → USD):** 1\u202fINR = **0.01136\u202fUSD**  \n(As of the latest update on 22\u202fOct\u202f2025.)\n\n**10\u202fINR to USD**\n\n\\[\n10 \\text{\u202fINR} \\times 0.01136 \\frac{\\text{USD}}{\\text{INR}} = 0.1136 \\text{\u202fUSD}\n\\]\n\nSo, 10\u202fINR is approximately **$0.11\u202fUSD**.'